In [157]:
import os 
import pandas as pd
from langchain.chains.summarize.refine_prompts import prompt_template
from openai import OpenAI, api_key
from dotenv import load_dotenv
from pathlib import Path
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI 
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma 
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from sympy.physics.units import temperature

from api import init_prompt


FileNotFoundError: [Errno 2] No such file or directory: 'data/回饋規則.xlsx'

In [17]:
load_dotenv()

True

In [117]:
init_prompt =  open('C:\Code\III_2024_golf\data\\rag_init_prompt.txt').read()
type(init_prompt)

str

'你是一位充滿教學熱忱的高爾夫球教練，我會給你以下資訊:\n1.某位學員在P2上桿(A)、P3上桿(A)、P4 頂點/轉換(F)、P5~6 下桿(I)、P7 擊球(T)的高爾夫擊球動作片段圖集，總共十張，上半部五張是學員正面的的照片，下半部五張是學員側面的圖片，每張圖片紅色點與綠色的線代表學員的骨架，白色輔助線代表球員所在位置 ，粉色的輔助線代表球員的集求姿態不該超越的位置，橙色的輔助線代表揮桿夾角\n2.學員的擊球數據，包含球速、發射角度、發射方向(球相對於目標線開始的初始方向。正發射方向表示球從目標右側開始，負發射方向表示球從目標左側開始)、飛行距離、ClubAngleFace(高爾夫球手將此稱為具有“開放”或“封閉”桿面。技術定義：在高爾夫球最大壓縮時，球桿面和高爾夫球接觸中心點的水平球桿面方向)、ClubAnglePath(高爾夫擊球的預期曲率(旋轉軸)的關鍵因素。假設中心接觸，球應該朝向面角彎曲並遠離球桿路徑(+5、0、-5))\n3.與教練的正面和側面姿勢在A、F、I、T四個階段各個關節的差異，格式為front:{{A:[肩膀,手肘,手腕,髖部,膝蓋,腳踝],F:[......}}side:{{A:[肩膀,手肘,手腕,髖部,膝蓋,腳踝],F:[......}}\n4.回饋規則\n5.Clubface = clubAnglePath - clubAngleFace，大於3代表在原本的球路上還會偏左，小於3代表在原本的球路上還會遍右，例如左飛球碰到Clubface 大於3就會變成左拉左曲球\n6.正確輸出建議模板:"......在我們全面的揮杆分析中，您的揮杆動作與教練大致相同，請繼續保持......"\n7.失誤輸出建議模板:"......在我們全面的揮杆分析中，您的揮杆動作與教練最相似的地方在於<階段>，請繼續保持，然而您的揮杆有幾個地方可以加強。在您的<階段>、考慮調整您的<關節>，試著<調整方法>，這將有助於......"\n\n\n\n你有以下幾項任務:\n1.觀察學員正面與側面圖片的骨架以及參考線，找到姿勢不正確的圖片\n2.判斷資訊3中與教練的姿勢差異是否有特別突出的地方，如果差異都不大，則判斷是那種正確球路，如果差異有特別突出的地方，則判斷是那種失誤球路\n3.每次分析都需要根據任務2從回饋規則內尋找最相似的原因，結合Clubface以及圖片

# RAG建構 

In [24]:
# 加载 Excel 文件
file_path = "C:\Code\III_2024_golf\data\回饋規則.xlsx"
data = pd.read_excel(file_path)

# 查看数据
print(data.head())

             球路類型  結果                        原因                      改善建議
0         Pull左飛球  失誤          上桿(P2~3)時，角度過於陡峭          上桿(P2~3)時，肩膀往右轉動
1         Pull左飛球  失誤                    桿頭頂點過高                    肩膀往右轉動
2         Pull左飛球  失誤  下桿角度過於陡峭，左手腕過度外展，肩關節伸展抬起  下桿時，左手臂打直、左手腕維持固定、肩膀自然旋轉
3         Pull左飛球  失誤            桿面關閉，擊球點位於球的外側       擊球時，左手腕維持固定，注意擊球點位置
4  Pull Hook左拉左曲球  失誤          上桿(P2~3)時，角度過於陡峭          上桿(P2~3)時，肩膀往右轉動


In [25]:
documents = [
    Document(
        page_content=" | ".join(map(str, row.values)), 
        metadata={"row_index": index}
    )
    for index, row in data.iterrows()
]

print(f"生成了 {len(documents)} 個文檔")
print(documents[0])

生成了 28 個文檔
page_content='Pull左飛球 | 失誤 | 上桿(P2~3)時，角度過於陡峭 | 上桿(P2~3)時，肩膀往右轉動' metadata={'row_index': 0}


In [29]:
# 使用 OpenAI Embeddings 对文档进行向量化
import chromadb
openai_api_key = os.getenv("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# 创建向量数据库
vectorstore = Chroma.from_documents(documents, embeddings)


In [161]:
import base64


def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [184]:


# 初始化生成模型
client = ChatOpenAI(api_key= openai_api_key,model = "chatgpt-4o-latest",temperature=0.2)

# 构建 RetrievalQA 链
retriever = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_messages([
    ('system', init_prompt+'{context}'),
    ('user', [
                {"type": "text", "text": "{input}"},
                {
                    "type": "image_url",
                    "image_url": {
                            "url": "data:image/jpeg;base64,'{image}'",
                            "detail": "high",
                        },
                },
            ],
        ),
    ])

document_chain = create_stuff_documents_chain(client, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# 用户提问
query = "球速:66.43、發射角度:7.198、發射方向:-6.172、飛行距離:179、ClubAngleFace:-15、ClubAnglePath:6.901front:{A: [0.0, 0.0, -0.0, 0.0, -0.0, -0.0], F: [-0.01, -0.008, 0.0, 0.048, -0.018, -0.011], I: [-0.019, -0.04, 0.184, -0.063, -0.041, -0.023], T: [0.002, 0.0, 0.0, -0.001, -0.001, -0.001]}side:{A: [0.002, -0.001, 0.001, -0.0, -0.002, 0.0], F: [0.001, 0.004, 0.0, -0.001, -0.003, 0.0], I: [0.217, -0.065, -0.013, -0.068, -0.054, -0.013], T: [-0.049, -0.043, 0.253, -0.083, -0.055, -0.013]}"

image = encode_image("C:\Code\III_2024_golf\data/combined/combined_186382.jpg")
response = retrieval_chain.invoke({
    'context': retriever,
    'image': image,
    'input': query,
})

print(response['answer'])


{
        "球路": "Pull左飛球",
        "原因": "Clubface數值為-15，ClubAnglePath為6.901，導致Clubface大於3，球路偏左；同時，從圖片觀察到P4頂點時肩膀旋轉不足，導致桿面過於封閉。",
        "建議": "在上桿至頂點(P4)時，專注於肩膀的旋轉，確保肩膀與髖部同步轉動，避免僅依靠手臂完成揮桿。可以嘗試在家中進行鏡子練習，觀察肩膀是否充分旋轉，並保持脊椎角度穩定，這將有助於改善桿面角度，減少左飛球的發生。"
}


In [99]:
from openai import OpenAI
import pandas as pd


# 讀取 CSV 檔案
csv_file_path = "data/回饋規則.xlsx"
data = pd.read_excel(csv_file_path)

# 將數據轉換為文字格式（或提取特定欄位）
text_representation = data.to_string()  

# 定義提示字串
prompt = f"""
{init_prompt}
以下是一些來自回饋規則檔案的數據：
{text_representation}

"""
image_path = "data/combined/combined_186382.jpg"
encoded_image = encode_image(image_path)


# 呼叫 OpenAI API
client = OpenAI()

completion = client.chat.completions.create(
    model="chatgpt-4o-latest",
    messages=[
        {"role": "system", "content": prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", 
                 "text": ""},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"https://cdn.discordapp.com/attachments/727209373612638362/1315702149514137763/combined_186382.jpg?ex=6768d95d&is=676787dd&hm=3e47443ea303edbc8e73e6994f2df268aacae3b1f731155554fa2dc58e7c9f2b&",
                        "detail": "high",
                    }
                },
            ],
        }
    ]
)

print(completion.choices[0].message.content)



{
    "球路": "Pull Hook左拉左曲球",
    "原因": "上桿(P2~3)時角度過於陡峭，且桿頭頂點過高，影響下桿軌跡。下桿過於陡峭，手腕過度彎曲；桿面過於閉合且擊球點位於球的外側。",
    "建議": "您的揮桿動作與教練最相似的地方在於P4頂點位置，請繼續保持。然而，建議您在上桿至P3時，肩膀轉動要保持穩定且避免過度，因此試著稍微減少手腕內收與保持桿面角度。下桿時，關注於肩膀與手臂的協調，讓左手腕固定且避免過度彎曲，並注意擊球點位置，以提升穩定性與準確度。此外，P2至P3過程中可緩慢練習，確保桿身與地面保持適當角度避免過陡。"
}
